# Results of simulation of different fairness policies

These experiments use accumulated deficits to try to ensure that applications always receive their computed allocation of GPU time, even in the event of new jobs coming in and old jobs finishing, by keeping track of the difference between the GPU time the application should have received, and the GPU time the application actually received. Allocation of jobs to GPUs is performed in a round-based fashion, with GPUs instructed to run jobs for a fixed interval of time on all GPUs.

# Import statements

In [1]:
# Imports for plotting.
from matplotlib import pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
from matplotlib.ticker import MultipleLocator
from pylab import *
import seaborn as sns
from matplotlib import rc
rc('text',
   usetex=True)
sns.set_style('ticks')
font = {
    'font.family':'Times New Roman',
    'font.weight': 200,
    'font.size': 10,
}
sns.set_style(font)
flatui = ['#002A5E', '#FD151B', '#8EBA42', '#348ABD', '#988ED5', '#BDB76B', '#8EBA42', '#FFB5B8']
sns.set_palette(flatui)
paper_rc = {
    'lines.linewidth': 2,
    'lines.markersize': 10,
    'mathtext.fontset': 'custom',
    'mathtext.rm': 'Times New Roman',
    'mathtext.bf': 'Times New Roman:bold',
}
sns.set_context("paper", font_scale=2,  rc=paper_rc)
current_palette = sns.color_palette()

In [2]:
# Other imports.
import json
import os
import re

# Get logfile paths

In [3]:
def get_logfile_paths_helper(directory_name):
    logfile_paths = []
    for root, _, file_names in os.walk(directory_name):
        if len(file_names) > 0:
            logfile_paths.extend(
                [os.path.join(root, file_name)
                 for file_name in file_names])
    return logfile_paths

def get_logfile_paths(directory_name):
    logfile_paths = []
    for logfile_path in get_logfile_paths_helper(directory_name):
        m = re.match(
            r'.*v100=(\d+)\.p100=(\d+)\.k80=(\d+)/(.*)/seed=(\d+)/'
             'lambda=(\d+\.\d+)\.log', logfile_path)
        v100s = int(m.group(1))
        p100s = int(m.group(2))
        k80s = int(m.group(3))
        policy = m.group(4)
        seed = int(m.group(5))
        l = float(m.group(6))
        logfile_paths.append((v100s, p100s, k80s, policy, seed,
                              l, logfile_path))
    return logfile_paths

# Munging functions

In [9]:
def prune(logfile_paths, v100s, p100s, k80s, policy, seed=None):
    if seed is None:
        return sorted([(x[5], x[6], x[4]) for x in logfile_paths
                       if x[0] == v100s and x[1] == p100s and
                       x[2] == k80s and x[3] == policy])
    else:
        return sorted([(x[5], x[6]) for x in logfile_paths
                       if x[0] == v100s and x[1] == p100s and
                       x[2] == k80s and x[3] == policy and
                       x[4] == seed])

In [14]:
def get_events(logfile_path):
    events = {}
    utilization = None
    with open(logfile_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            m = re.match(r'(\d+\.\d+).*scheduled.*Job ID: (\d+)\t'
                          'Worker type: (.*)\tWorker ID\(s\): ([\d+\,]*\d+).*',
                         line)
            if m is not None:
                start_timestamp = float(m.group(1))
                job_id = int(m.group(2))
                worker_type = m.group(3)
                worker_ids = [int(x) for x in m.group(4).split(',')]
                for worker_id in worker_ids:
                    if worker_id not in events:
                        events[worker_id] = []
                    events[worker_id].append([job_id, worker_type,
                                              start_timestamp,
                                              None])
            m = re.match(r'(\d+\.\d+).*scheduled.*Job ID: \((\d+), (\d+)\)\t'
                          'Worker type: (.*)\tWorker ID\(s\): ([\d+\,]*\d+).*',
                         line)
            if m is not None:
                start_timestamp = float(m.group(1))
                job_id1 = int(m.group(2))
                job_id2 = int(m.group(3))
                worker_type = m.group(4)
                worker_ids = [int(x) for x in m.group(5).split(',')]
                for worker_id in worker_ids:
                    if worker_id not in events:
                        events[worker_id] = []
                    events[worker_id].append([(job_id1, job_id2), worker_type,
                                              start_timestamp,
                                              None])
            
            m = re.match(r'(\d+\.\d+).*succeeded.*Job ID: (\d+)\t'
                          'Worker type: (.*)\tWorker ID: (\d+).*',
                         line)
            if m is not None:
                end_timestamp = float(m.group(1))
                job_id = int(m.group(2))
                worker_type = m.group(3)
                worker_id = int(m.group(4))
                if worker_id not in events:
                    continue
                assert(events[worker_id][-1][0] == job_id and
                       events[worker_id][-1][1] == worker_type and
                       events[worker_id][-1][3] is None)
                events[worker_id][-1][3] = end_timestamp
                
            m = re.match(r'(\d+\.\d+).*succeeded.*Job ID: \((\d+), (\d+)\)\t'
                          'Worker type: (.*)\tWorker ID: (\d+).*',
                         line)
            if m is not None:
                end_timestamp = float(m.group(1))
                job_id1 = int(m.group(2))
                job_id2 = int(m.group(3))
                worker_type = m.group(4)
                worker_id = int(m.group(5))
                if worker_id not in events:
                    continue
                assert(events[worker_id][-1][0] == (job_id1, job_id2) and
                       events[worker_id][-1][1] == worker_type and
                       events[worker_id][-1][3] is None)
                events[worker_id][-1][3] = end_timestamp
                
            m = re.match(r'Cluster utilization: (\d+\.\d+)', line)
            if m is not None:
                utilization = float(m.group(1)) * 100.

    return events, utilization


In [6]:
worker_type_map = {
    'k80': 0,
    'p100': 1,
    'v100': 2,
}
def get_num_promotions_per_job(logfile_path):
    events, _ = get_events(logfile_path)
    per_job_timeline = {}
    num_promotions_per_job = {}
    for worker_id in events:
        for i, (job_id, worker_type, start_time, _) in enumerate(events[worker_id]):
            worker_type_int = worker_type_map[worker_type]
            if type(job_id) == tuple:
                for job_id_ in job_id:
                    if job_id_ not in per_job_timeline:
                        per_job_timeline[job_id_] = []
                    per_job_timeline[job_id_].append((worker_type_int, start_time))
            else:
                if job_id not in per_job_timeline:
                    per_job_timeline[job_id] = []
                per_job_timeline[job_id].append((worker_type_int, start_time))
    for job_id in per_job_timeline:
        per_job_timeline[job_id].sort(key=lambda x: x[1])
    for job_id in per_job_timeline:
        num_promotions_per_job[job_id] = 0
        for i, (worker_type, _) in enumerate(per_job_timeline[job_id]):
            if i == 0: continue
            if worker_type != per_job_timeline[job_id][i-1][0]:
                num_promotions_per_job[job_id] += 1
        if num_promotions_per_job[job_id] > 2:
            print(per_job_timeline[job_id])
            break
    return num_promotions_per_job

In [7]:
def sweep_get_num_promotions_per_job(relevant_logfile_paths):
    for lam, logfile_path, seed in relevant_logfile_paths:
        num_promotions_per_job = get_num_promotions_per_job(logfile_path)
        all_num_promotions = [num_promotions_per_job[job_id] for job_id in num_promotions_per_job]
        print('Lambda=%f, seed=%d: Average number of promotions: %f (stddev %f)' % (
            lam,
            seed,
            np.mean(all_num_promotions),
            np.std(all_num_promotions)))
        print('Distinct all_num_promotions:', set(all_num_promotions))
        print()

# Perform sweep

In [11]:
logfile_paths = sorted(get_logfile_paths(
    "/lfs/1/deepak/gpusched/scheduler/logs/multigpu_support_singlegpu/"))

In [10]:
relevant_logfile_paths = prune(logfile_paths, 8, 8, 8, 'fifo')
relevant_logfile_paths.reverse()
sweep_get_num_promotions_per_job(relevant_logfile_paths)

Lambda=50400.000000, seed=0: Average number of promotions: 0.000000 (stddev 0.000000)
Distinct all_num_promotions: {0}

Lambda=25200.000000, seed=0: Average number of promotions: 0.000000 (stddev 0.000000)
Distinct all_num_promotions: {0}

Lambda=16800.000000, seed=0: Average number of promotions: 0.000000 (stddev 0.000000)
Distinct all_num_promotions: {0}

Lambda=12600.000000, seed=0: Average number of promotions: 0.000000 (stddev 0.000000)
Distinct all_num_promotions: {0}

Lambda=10080.000000, seed=0: Average number of promotions: 0.000000 (stddev 0.000000)
Distinct all_num_promotions: {0}

Lambda=8400.000000, seed=0: Average number of promotions: 0.000000 (stddev 0.000000)
Distinct all_num_promotions: {0}



In [11]:
relevant_logfile_paths = prune(logfile_paths, 8, 8, 8, 'fifo_perf')
relevant_logfile_paths.reverse()
sweep_get_num_promotions_per_job(relevant_logfile_paths)

Lambda=50400.000000, seed=0: Average number of promotions: 0.004396 (stddev 0.066153)
Distinct all_num_promotions: {0, 1}

Lambda=25200.000000, seed=0: Average number of promotions: 0.105526 (stddev 0.307878)
Distinct all_num_promotions: {0, 1, 2}

Lambda=16800.000000, seed=0: Average number of promotions: 0.356958 (stddev 0.512432)
Distinct all_num_promotions: {0, 1, 2}

Lambda=12600.000000, seed=0: Average number of promotions: 0.677458 (stddev 0.623631)
Distinct all_num_promotions: {0, 1, 2}

Lambda=10080.000000, seed=0: Average number of promotions: 0.984224 (stddev 0.731673)
Distinct all_num_promotions: {0, 1, 2}

Lambda=8400.000000, seed=0: Average number of promotions: 1.134880 (stddev 0.768976)
Distinct all_num_promotions: {0, 1, 2}



In [12]:
logfile_paths = sorted(get_logfile_paths(
    "/lfs/1/deepak/gpusched/scheduler/logs/multigpu_support_multigpu/"))

In [13]:
relevant_logfile_paths = prune(logfile_paths, 8, 8, 8, 'fifo')
relevant_logfile_paths.reverse()
sweep_get_num_promotions_per_job(relevant_logfile_paths)

Lambda=50400.000000, seed=0: Average number of promotions: 0.000000 (stddev 0.000000)
Distinct all_num_promotions: {0}

Lambda=25200.000000, seed=0: Average number of promotions: 0.000000 (stddev 0.000000)
Distinct all_num_promotions: {0}

Lambda=16800.000000, seed=0: Average number of promotions: 0.000000 (stddev 0.000000)
Distinct all_num_promotions: {0}

Lambda=12600.000000, seed=0: Average number of promotions: 0.000000 (stddev 0.000000)
Distinct all_num_promotions: {0}

Lambda=10080.000000, seed=0: Average number of promotions: 0.000000 (stddev 0.000000)
Distinct all_num_promotions: {0}

Lambda=8400.000000, seed=0: Average number of promotions: 0.000000 (stddev 0.000000)
Distinct all_num_promotions: {0}



In [ ]:
relevant_logfile_paths = prune(logfile_paths, 8, 8, 8, 'fifo_perf')
relevant_logfile_paths.reverse()
sweep_get_num_promotions_per_job(relevant_logfile_paths)

[(0, 56442140.019466974), (0, 56442140.019466974), (0, 56442140.019466974), (0, 56442140.019466974), (0, 56444060.01883911), (0, 56444060.01883911), (0, 56444060.01883911), (0, 56444060.01883911), (0, 56445980.01821125), (0, 56445980.01821125), (0, 56445980.01821125), (0, 56445980.01821125), (0, 56447900.01758339), (0, 56447900.01758339), (0, 56447900.01758339), (0, 56447900.01758339), (0, 56449820.01695553), (0, 56449820.01695553), (0, 56449820.01695553), (0, 56449820.01695553), (0, 56451740.01632767), (0, 56451740.01632767), (0, 56451740.01632767), (0, 56451740.01632767), (0, 56453660.01569981), (0, 56453660.01569981), (0, 56453660.01569981), (0, 56453660.01569981), (0, 56455580.01507195), (0, 56455580.01507195), (0, 56455580.01507195), (0, 56455580.01507195), (0, 56457500.01444409), (0, 56457500.01444409), (0, 56457500.01444409), (0, 56457500.01444409), (0, 56459420.01381623), (0, 56459420.01381623), (0, 56459420.01381623), (0, 56459420.01381623), (0, 56461340.01318837), (0, 5646134

[(0, 804478.5777866368), (0, 806398.5764194897), (0, 808318.5750523426), (0, 810238.5736851955), (0, 812158.5723180484), (0, 814078.5709509014), (0, 815998.5695837543), (0, 817918.5682166072), (0, 819838.5668494601), (0, 821758.565482313), (1, 823678.564115166), (1, 825598.5627480189), (1, 827518.5613808718), (1, 829438.5600137247), (1, 831358.5586465776), (1, 833278.5572794306), (1, 835198.5559122835), (1, 837118.5523774171), (1, 839038.5488425507), (1, 840958.5453076843), (1, 842878.541772818), (1, 844798.5382379516), (1, 846718.5347030852), (1, 848638.5311682188), (1, 850558.5276333524), (1, 852478.524098486), (1, 854398.5205636197), (1, 856318.5170287533), (1, 858238.5134938869), (1, 860158.5099590205), (1, 862078.5064241542), (1, 863998.5028892878), (1, 865918.4993544214), (1, 867838.495819555), (1, 869758.4922846886), (1, 871678.4887498223), (1, 873598.4852149559), (1, 875518.4816800895), (1, 877438.4781452231), (1, 879358.4746103567), (1, 881278.4710754903), (1, 883198.467540624